# Wit.ai

Notebook for ingest and train a model in Wit.ai. In the last section you could test your model via API.

In [ ]:
import requests as req
import pandas as pd
from intentbasedbot.config import *

## Training data

In [ ]:
utterances_df = pd.read_csv('data/snips_utterances.csv')

In [ ]:
utterances_df.info()

In [ ]:
# Values of column Intention
snips_intents = ['BookRestaurant', 'GetWeather', 'PlayMusic', 'AddToPlaylist', 'RateBook', 'SearchScreeningEvent', 'SearchCreativeWork']

In [ ]:
language = 'Spanish'

In [ ]:
utterances_dic = {}
for snips_intent in snips_intents:
    print('Fetching intents of', snips_intent)
    utterances_dic[snips_intent] = utterances_df[utterances_df['Intention'] == snips_intent][language].tolist()

In [ ]:
list(utterances_dic.keys()) == snips_intents

In [ ]:
len(utterances_dic['BookRestaurant'])

## Posting the training data

In [ ]:
# Checking builtin entities
endpoint = f'{WIT_URL}{WIT_ENTITIES_URI}'
params = {'v': WIT_VERSION}
response = req.get(endpoint, params=params, headers=WIT_HEADERS,)
response.json()

In [ ]:
def build_sample(intent_utterance, intent):
    return {
        "text": intent_utterance,
        "entities": [
          {
            "entity": "intent",
            "value": intent
          }
        ]
      }

In [ ]:
# Post all intent samples - Already runned
endpoint = f'{WIT_URL}{WIT_SAMPLES_URI}'
params = {'v': WIT_VERSION}

for intent in utterances_dic.keys():
    samples = [build_sample(intent_utterance, intent) for intent_utterance in utterances_dic[intent]]
    print('Samples of', intent, len(samples))    
    
    response = req.post(endpoint, params=params, headers=WIT_HEADERS, json=samples)
    print('Response', response.text)

In [ ]:
# Checking sample push
endpoint = f'{WIT_URL}{WIT_ENTITIES_URI}/intent'
params = {'v': WIT_VERSION}
response = req.get(endpoint, params=params, headers=WIT_HEADERS,)
wit_intents = response.json()['values']

In [ ]:
for wit_intent in wit_intents:
    print('Intent', wit_intent['value'], 'Samples', len(wit_intent['expressions']))

## Testing

In [ ]:
query = 'Quiero salir a comer mañana'

context = {
  "reference_time": "2018-07-14T18:08:00",
  "timezone": "America/Argentina/Buenos_Aires"
}
params = {'v': WIT_VERSION, 'q': query, 'verbose': True, 'n': 3, 'context': context}
endpoint = f'{WIT_URL}{WIT_MESSAGE_URI}'

In [ ]:
response = req.get(endpoint, params=params, headers=WIT_HEADERS,)
response.json()